# 1. Prepare Data

In [ ]:
import pandas as pd

# Load the 2023 profit data
file_path_2023 = 'Profits_2023.xlsx'
months_2023 = ['Jan23', 'Feb23', 'mar23', 'Apr23', 'may23', 'Jun23', 'Jul23', 'Aug23', 'Sep23', 'Oct23', 'Nov23', 'Dec23']

# Extract profit data for 2023 from the "กำไรหลังAudit/Wo" column
df_2023 = pd.concat([pd.read_excel(file_path_2023, sheet_name=month)[['Store Name', 'กำไรหลังAudit/Wo']].rename(columns={'กำไรหลังAudit/Wo': month})
                     for month in months_2023], axis=1)

# Drop duplicate columns and reset index to avoid duplicate 'Store Name'
df_2023 = df_2023.loc[:, ~df_2023.columns.duplicated()]

# Load the 2024 profit data
file_path_2024 = 'Profits_2024.xlsx'

# Extract January and February 2024 profit data
df_2024_jan_feb = pd.concat([pd.read_excel(file_path_2024, sheet_name=month)[['Store Name', 'กำไรหลังAudit/Wo']].rename(columns={'กำไรหลังAudit/Wo': month})
                             for month in ['มค', 'กพ']], axis=1)

# Drop duplicate columns
df_2024_jan_feb = df_2024_jan_feb.loc[:, ~df_2024_jan_feb.columns.duplicated()]

# Load March to June 2024 data with correct header rows
df_2024_mar = pd.read_excel(file_path_2024, sheet_name='มีค', header=1)[['Store Name', 'กำไรหลังAudit/Wo']].rename(columns={'กำไรหลังAudit/Wo': 'มีค'})
df_2024_apr = pd.read_excel(file_path_2024, sheet_name='เมย', header=1)[['Store Name', 'กำไรระดับสาขารวมโบนัส+ปันส่วนจาก 7-11']].rename(columns={'กำไรระดับสาขารวมโบนัส+ปันส่วนจาก 7-11': 'เมย'})
df_2024_may = pd.read_excel(file_path_2024, sheet_name='พค', header=1)[['Store Name', 'กำไรระดับสาขารวมโบนัส+ปันส่วนจาก 7-11']].rename(columns={'กำไรระดับสาขารวมโบนัส+ปันส่วนจาก 7-11': 'พค'})
df_2024_jun = pd.read_excel(file_path_2024, sheet_name='มิย', header=2)[['Store Name', 'กำไรระดับสาขารวมโบนัส+ปันส่วนจาก 7-11']].rename(columns={'กำไรระดับสาขารวมโบนัส+ปันส่วนจาก 7-11': 'มิย'})

# Combine March to June data
df_2024_mar_jun = pd.merge(df_2024_mar, df_2024_apr, on='Store Name', how='outer')
df_2024_mar_jun = pd.merge(df_2024_mar_jun, df_2024_may, on='Store Name', how='outer')
df_2024_mar_jun = pd.merge(df_2024_mar_jun, df_2024_jun, on='Store Name', how='outer')

# Drop duplicate columns in March to June data
df_2024_mar_jun = df_2024_mar_jun.loc[:, ~df_2024_mar_jun.columns.duplicated()]

# Combine the January to February data with the March to June data
df_2024_combined = pd.merge(df_2024_jan_feb, df_2024_mar_jun, on='Store Name', how='outer')

# Drop duplicate columns in the combined 2024 data
df_2024_combined = df_2024_combined.loc[:, ~df_2024_combined.columns.duplicated()]

# Now, merge the 2023 profit data with the 2024 combined data
combined_profits = pd.merge(df_2023, df_2024_combined, on='Store Name', how='outer')

# Ensure no duplicate columns are present
combined_profits = combined_profits.loc[:, ~combined_profits.columns.duplicated()]

# Update the month names in the dataframe to follow the format "Jan2023", "Feb2023", etc.
month_mapping = {
    'Jan23': 'Jan2023', 'Feb23': 'Feb2023', 'mar23': 'Mar2023', 'Apr23': 'Apr2023',
    'may23': 'May2023', 'Jun23': 'Jun2023', 'Jul23': 'Jul2023', 'Aug23': 'Aug2023',
    'Sep23': 'Sep2023', 'Oct23': 'Oct2023', 'Nov23': 'Nov2023', 'Dec23': 'Dec2023',
    'มค': 'Jan2024', 'กพ': 'Feb2024', 'มีค': 'Mar2024', 'เมย': 'Apr2024',
    'พค': 'May2024', 'มิย': 'Jun2024'
}

# Rename the columns to reflect the new format
combined_profits = combined_profits.rename(columns=month_mapping)

# Remove rows where 'Store Name' is NaN or the row has missing data
combined_profits = combined_profits.dropna(subset=['Store Name']).reset_index(drop=True)

# Re-export the cleaned data to an Excel file
output_path = 'Cleaned_Combined_Profit_Data_2023_2024.xlsx'
combined_profits.to_excel(output_path, index=False)

output_path

# You can save or further process the combined_profits dataframe
combined_profits


# Visualize Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Load the cleaned dataset with Thai encoding
file_path = 'Cleaned_Combined_Profit_Data_2023_2024.xlsx'
data = pd.read_excel(file_path)

# Ensure Thai fonts are available
# You can specify the path to a Thai font if necessary, e.g., 'THSarabunNew.ttf'
font_path = 'THSarabunNew Bold.ttf'  # Example for Linux
# For Windows: font_path = 'C:/Windows/Fonts/THSarabunNew.ttf'
thai_font = fm.FontProperties(fname=font_path)

# Extract store names and monthly profit columns
store_names = data['Store Name']
profit_columns = data.columns[1:]  # All columns from Jan2023 to Jun2024

# Plot profits for each store
plt.figure(figsize=(12, 8))

# Loop through each store and plot its profits
for i, store in enumerate(store_names):
    profits = data.iloc[i, 1:].values  # Get profits for this store across all months
    plt.plot(profit_columns, profits, label=store)

# Customize the plot
plt.title('Monthly Profits for All Stores (Jan2023 to Jun2024)', fontsize=14, fontproperties=thai_font)
plt.xlabel('Month', fontsize=12, fontproperties=thai_font)
plt.ylabel('Profit', fontsize=12, fontproperties=thai_font)
plt.xticks(rotation=45, fontproperties=thai_font)
plt.legend(loc='best', bbox_to_anchor=(1, 1), prop=thai_font)  # Puts legend outside the plot
plt.tight_layout()

# Show the plot
plt.show()


# 2. Finding best model for individual stores

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error as mape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from scipy import stats
from prophet import Prophet
import os

# Load the cleaned dataset
file_path = 'Cleaned_Combined_Profit_Data_2023_2024.xlsx'
data = pd.read_excel(file_path)

# Output file for incremental saving
output_file = 'store_forecasting_best_models.csv'

# Check if the output file already exists
if os.path.exists(output_file):
    results_df = pd.read_csv(output_file)
    processed_stores = set(results_df['Store Name'].values)
else:
    results_df = pd.DataFrame()
    processed_stores = set()

# Function to create dataset for time series
def create_dataset(dataset, look_back=3):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# LSTM model
def build_lstm(look_back):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(look_back, 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# GRU model
def build_gru(look_back):
    model = Sequential()
    model.add(GRU(100, return_sequences=True, input_shape=(look_back, 1)))
    model.add(Dropout(0.2))
    model.add(GRU(50))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# CNN-LSTM model
def build_cnn_lstm(look_back):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(look_back, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(25))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# RandomForest and XGBoost models
def train_tree_model(model, X_train, y_train, X_test):
    # Remove any rows with NaN values
    X_train, y_train = X_train.reshape(-1, look_back), y_train
    mask = ~np.isnan(X_train).any(axis=1) & ~np.isnan(y_train)
    X_train = X_train[mask]
    y_train = y_train[mask]

    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    return predictions

# Hyperparameter tuning grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

# Prophet Forecasting
def build_prophet(train_data):
    df = pd.DataFrame({'ds': train_data.index, 'y': train_data.values})
    model = Prophet()
    model.fit(df)
    future = model.make_future_dataframe(periods=len(train_data))
    forecast = model.predict(future)
    return forecast['yhat'].values

# Iterate over each store
for store_name in data['Store Name'].unique():
    if store_name in processed_stores:
        print(f"Skipping {store_name}, already processed.")
        continue

    # Extract data for the current store
    store_data = data[data['Store Name'] == store_name].iloc[0, 1:].values
    store_series = pd.Series(store_data, index=pd.date_range(start='2023-01-01', periods=len(store_data), freq='M'))

    # Train-test split (80% train, 20% test)
    train_size = int(len(store_series) * 0.8)
    train_data = store_series[:train_size]
    test_data = store_series[train_size:]

    # Rescale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))

    look_back = 3
    X_train, y_train = create_dataset(train_scaled, look_back)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # Prepare test data
    test_scaled = scaler.transform(test_data.values.reshape(-1, 1))
    X_test, y_test = create_dataset(test_scaled, look_back)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Dictionary to store metrics for each model
    metrics = {}

    # Test LSTM model
    lstm_model = build_lstm(look_back)
    lstm_model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)
    lstm_predictions = scaler.inverse_transform(lstm_model.predict(X_test))
    lstm_mape = np.mean(np.abs((test_data[:len(lstm_predictions)] - lstm_predictions[:, 0]) / test_data[:len(lstm_predictions)])) * 100
    metrics['LSTM'] = lstm_mape

    # Test GRU model
    gru_model = build_gru(look_back)
    gru_model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)
    gru_predictions = scaler.inverse_transform(gru_model.predict(X_test))
    gru_mape = np.mean(np.abs((test_data[:len(gru_predictions)] - gru_predictions[:, 0]) / test_data[:len(gru_predictions)])) * 100
    metrics['GRU'] = gru_mape

    # Test CNN-LSTM model
    cnn_lstm_model = build_cnn_lstm(look_back)
    cnn_lstm_model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)
    cnn_lstm_predictions = scaler.inverse_transform(cnn_lstm_model.predict(X_test))
    cnn_lstm_mape = np.mean(np.abs((test_data[:len(cnn_lstm_predictions)] - cnn_lstm_predictions[:, 0]) / test_data[:len(cnn_lstm_predictions)])) * 100
    metrics['CNN-LSTM'] = cnn_lstm_mape

    # Test RandomForest model
    rf_model = RandomForestRegressor()
    # Adjust number of splits based on available data
    n_splits = min(3, len(X_train) - look_back)  # Ensure n_splits is not greater than available samples
    if n_splits > 1:
        grid_search = GridSearchCV(rf_model, param_grid, scoring='neg_mean_absolute_percentage_error', cv=TimeSeriesSplit(n_splits=n_splits))
        # Reshape and drop NaNs for RandomForest training
        X_train_reshaped = X_train.reshape(-1, look_back)
        mask_rf = ~np.isnan(X_train_reshaped).any(axis=1) & ~np.isnan(y_train)
        X_train_rf = X_train_reshaped[mask_rf]
        y_train_rf = y_train[mask_rf]

        if len(X_train_rf) > 0 and len(y_train_rf) > 0:
            grid_search.fit(X_train_rf, y_train_rf)
            rf_predictions = grid_search.predict(X_test.reshape(-1, look_back))
            rf_mape = mape(y_test, rf_predictions) * 100
            metrics['RandomForest'] = rf_mape
        else:
            print(f"Skipping RandomForest for {store_name} due to insufficient data.")
    else:
        print(f"Skipping RandomForest for {store_name} due to insufficient data for cross-validation.")

    # Test XGBoost model
    xgb_model = XGBRegressor()
    if n_splits > 1:
        grid_search_xgb = GridSearchCV(xgb_model, param_grid, scoring='neg_mean_absolute_percentage_error', cv=TimeSeriesSplit(n_splits=n_splits))
        # Reshape and drop NaNs for XGBoost training
        mask_xgb = ~np.isnan(X_train_reshaped).any(axis=1) & ~np.isnan(y_train)
        X_train_xgb = X_train_reshaped[mask_xgb]
        y_train_xgb = y_train[mask_xgb]

        if len(X_train_xgb) > 0 and len(y_train_xgb) > 0:
            grid_search_xgb.fit(X_train_xgb, y_train_xgb)
            xgb_predictions = grid_search_xgb.predict(X_test.reshape(-1, look_back))
            xgb_mape = mape(y_test, xgb_predictions) * 100
            metrics['XGBoost'] = xgb_mape
        else:
            print(f"Skipping XGBoost for {store_name} due to insufficient data.")
    else:
        print(f"Skipping XGBoost for {store_name} due to insufficient data for cross-validation.")

    # Test Prophet model
    prophet_predictions = build_prophet(train_data)
    prophet_mape = np.mean(np.abs((test_data.values - prophet_predictions[-len(test_data):]) / test_data.values)) * 100
    metrics['Prophet'] = prophet_mape

    # Choose the best model based on the lowest MAPE
    best_model = min(metrics, key=metrics.get)
    best_mape = metrics[best_model]

    # Store the best result for the store
    new_result_df = pd.DataFrame([{
        'Store Name': store_name,
        'Best Model': best_model,
        'MAPE (%)': best_mape
    }])
    
    # Append to the main DataFrame and save
    results_df = pd.concat([results_df, new_result_df], ignore_index=True)
    results_df.to_csv(output_file, index=False)

    print(f"Processed and saved {store_name} with best model: {best_model} and MAPE: {best_mape}%")

# Final output
print("All stores processed and saved.")


# Finding average MAPE value

In [ ]:
import pandas as pd

# Load the CSV file
file_path = 'store_forecasting_best_models.csv'  # Make sure this is the correct path on your system
data = pd.read_csv(file_path)

# Display the first few rows to verify the data
print(data.head())

# Define a threshold for outliers
threshold = 100  # Exclude stores with MAPE > 200%
data_filtered = data[data['MAPE (%)'] <= threshold]

# Summary statistics for MAPE without outliers
mape_summary = data_filtered['MAPE (%)'].describe()
print("\nMAPE Summary Statistic:\n", mape_summary)

# Calculate and print overall average MAPE without outliers
average_mape = data_filtered['MAPE (%)'].mean()
print(f"\nAverage MAPE across all stores: {average_mape:.2f}%")

# If you want to filter for stores with high MAPE (e.g., above 50%)
high_mape_stores = data_filtered[data_filtered['MAPE (%)'] > 50]
print("\nStores with MAPE greater than 50%:\n", high_mape_stores)

# 3. Outlier Detection and Smoothing Code

In [ ]:
import pandas as pd

# Load the combined profits file and the best model information file
profits_file_path = 'Cleaned_Combined_Profit_Data_2023_2024.xlsx'
best_model_file_path = 'store_forecasting_best_models.csv'

# Load the data
profits_data = pd.read_excel(profits_file_path)
best_model_data = pd.read_csv(best_model_file_path)

# Set a threshold for high MAPE values
high_mape_threshold = 100

# Identify outlier stores based on MAPE values
outlier_stores = best_model_data[best_model_data['MAPE (%)'] > high_mape_threshold]['Store Name'].unique()

# Function to apply smoothing to the store data using a rolling average
def smooth_data(data, window_size=3):
    return data.rolling(window=window_size, min_periods=1).mean()

# Apply smoothing and save the updated dataset for outlier stores
for store_name in outlier_stores:
    store_data = profits_data[profits_data['Store Name'] == store_name].iloc[0, 1:]
    store_series = pd.Series(store_data.values, index=pd.date_range(start='2023-01-01', periods=len(store_data), freq='M'))
    smoothed_series = smooth_data(store_series)
    
    # Update the profits data with smoothed values for the outlier store
    profits_data.loc[profits_data['Store Name'] == store_name, profits_data.columns[1:]] = smoothed_series.values
    print(f"Applied smoothing for store: {store_name}")

# Save the smoothed dataset (optional, if you want to keep track of it separately)
profits_data.to_excel('Smoothed_Combined_Profit_Data_2023_2024.xlsx', index=False)
print("Outlier detection and smoothing complete.")


# 4. Prediction Code Using Smoothed Data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from prophet import Prophet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os

# Load the smoothed profits file and the best model information file
profits_file_path = 'Smoothed_Combined_Profit_Data_2023_2024.xlsx'
best_model_file_path = 'store_forecasting_best_models.csv'

# Load the data
profits_data = pd.read_excel(profits_file_path)
best_model_data = pd.read_csv(best_model_file_path)

# Output file for incremental saving
output_file = 'store_forecast_results_new.csv'

# Check if the output file already exists
if os.path.exists(output_file):
    forecast_df = pd.read_csv(output_file)
    processed_stores = set(forecast_df['Store Name'].unique())
else:
    forecast_df = pd.DataFrame()
    processed_stores = set()

# Function to create dataset
def create_dataset(dataset, look_back=3):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# Function to build and compile LSTM model
def build_lstm(look_back):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(look_back, 1), kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, kernel_regularizer=l2(0.001)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# Function to build and compile GRU model
def build_gru(look_back):
    model = Sequential()
    model.add(GRU(100, return_sequences=True, input_shape=(look_back, 1), kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.2))
    model.add(GRU(50, kernel_regularizer=l2(0.001)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# Function to build and compile CNN-LSTM model
def build_cnn_lstm(look_back):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(look_back, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(25))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# Function to build and fit RandomForest model
def build_random_forest(X_train, y_train):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    return model

# Function to build and fit XGBoost model
def build_xgboost(X_train, y_train):
    model = XGBRegressor()
    model.fit(X_train, y_train)
    return model

# Function to build and fit Prophet model
def build_prophet(train_data):
    df = pd.DataFrame({'ds': train_data.index, 'y': train_data.values})
    model = Prophet()
    model.fit(df)
    future = model.make_future_dataframe(periods=len(train_data))
    forecast = model.predict(future)
    return forecast['yhat'].values

# Iterate over each store to forecast using the best model
for store_name in profits_data['Store Name'].unique():
    if store_name in processed_stores:
        print(f"Skipping {store_name}, already processed.")
        continue

    # Extract the best model for the store
    best_model = best_model_data.loc[best_model_data['Store Name'] == store_name, 'Best Model'].values[0]

    # Extract the profit data for the current store
    store_data = profits_data[profits_data['Store Name'] == store_name].iloc[0, 1:].values
    store_series = pd.Series(store_data, index=pd.date_range(start='2023-01-01', periods=len(store_data), freq='M'))

    # Train-test split (80% train, 20% test)
    train_size = int(len(store_series) * 0.8)
    train_data = store_series[:train_size]
    test_data = store_series[train_size:]

    # Rescale the data for deep learning models
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))

    look_back = 3
    X_train, y_train = create_dataset(train_scaled, look_back)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # Prepare the test data
    test_scaled = scaler.transform(test_data.values.reshape(-1, 1))
    X_test, y_test = create_dataset(test_scaled, look_back)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Initialize the appropriate model based on the best model information
    if best_model == 'LSTM':
        model = build_lstm(look_back)
        model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)
        predictions_scaled = model.predict(X_test)
        predictions = scaler.inverse_transform(predictions_scaled).flatten()
    
    elif best_model == 'GRU':
        model = build_gru(look_back)
        model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)
        predictions_scaled = model.predict(X_test)
        predictions = scaler.inverse_transform(predictions_scaled).flatten()

    elif best_model == 'CNN-LSTM':
        model = build_cnn_lstm(look_back)
        model.fit(X_train, y_train, epochs=50, batch_size=5, verbose=0)
        predictions_scaled = model.predict(X_test)
        predictions = scaler.inverse_transform(predictions_scaled).flatten()

    elif best_model == 'RandomForest':
        X_train_rf, y_train_rf = create_dataset(train_data.values.reshape(-1, 1), look_back)
        X_test_rf, y_test_rf = create_dataset(test_data.values.reshape(-1, 1), look_back)
        model = build_random_forest(X_train_rf, y_train_rf)
        predictions = model.predict(X_test_rf)

    elif best_model == 'XGBoost':
        X_train_xgb, y_train_xgb = create_dataset(train_data.values.reshape(-1, 1), look_back)
        X_test_xgb, y_test_xgb = create_dataset(test_data.values.reshape(-1, 1), look_back)
        model = build_xgboost(X_train_xgb, y_train_xgb)
        predictions = model.predict(X_test_xgb)

    elif best_model == 'Prophet':
        predictions = build_prophet(train_data)[-len(test_data):]

    else:
        print(f"Unknown model type for store: {store_name}")
        continue

    # Store the forecast results in a DataFrame
    store_forecast_df = pd.DataFrame({
        'Store Name': [store_name] * len(predictions),
        'Date': test_data.index[:len(predictions)],
        'Actual': test_data.values[:len(predictions)],
        'Predicted': predictions
    })

    # Append and save incrementally
    forecast_df = pd.concat([forecast_df, store_forecast_df], ignore_index=True)
    forecast_df.to_csv(output_file, index=False)

    print(f"Processed and saved {store_name}")

# Final output
print("All stores processed.")


# 5. Future Predictions for Each Store

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from prophet import Prophet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Conv1D, MaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import os

# Load the smoothed profits file and the best model information file
profits_file_path = 'Smoothed_Combined_Profit_Data_2023_2024.xlsx'
best_model_file_path = 'store_forecasting_best_models.csv'

# Load the data
profits_data = pd.read_excel(profits_file_path)
best_model_data = pd.read_csv(best_model_file_path)

# Define the number of future months to predict
future_months = 6  # For example, predicting 6 future months

# Function to create dataset for LSTM/GRU/CNN-LSTM
def create_dataset(dataset, look_back=3):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# LSTM model
def build_lstm(look_back):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(look_back, 1), kernel_regularizer=l2(0.001)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, kernel_regularizer=l2(0.001)))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# GRU model
def build_gru(look_back):
    model = Sequential()
    model.add(GRU(100, return_sequences=True, input_shape=(look_back, 1)))
    model.add(Dropout(0.2))
    model.add(GRU(50))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# CNN-LSTM model
def build_cnn_lstm(look_back):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(look_back, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(LSTM(50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(25))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# RandomForest model
def build_random_forest(X_train, y_train):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    return model

# XGBoost model
def build_xgboost(X_train, y_train):
    model = XGBRegressor()
    model.fit(X_train, y_train)
    return model

# Prophet model
def build_prophet(train_data):
    df = pd.DataFrame({'ds': train_data.index, 'y': train_data.values})
    model = Prophet()
    model.fit(df)
    future = model.make_future_dataframe(periods=future_months)
    forecast = model.predict(future)
    return forecast['yhat'].values

# Dictionary to store future predictions for all stores
future_predictions = {}

# Iterate over each store to forecast future months using the best model
for store_name in profits_data['Store Name'].unique():
    # Extract the best model for the store
    best_model = best_model_data.loc[best_model_data['Store Name'] == store_name, 'Best Model'].values[0]

    # Extract the profit data for the current store
    store_data = profits_data[profits_data['Store Name'] == store_name].iloc[0, 1:].values
    store_series = pd.Series(store_data, index=pd.date_range(start='2023-01-01', periods=len(store_data), freq='M'))

    # Rescale the data for LSTM/GRU/CNN-LSTM
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(store_series.values.reshape(-1, 1))

    look_back = 3
    X, y = create_dataset(scaled_data, look_back)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Initialize the appropriate model based on the best model information
    if best_model == 'LSTM':
        model = build_lstm(look_back)
        model.fit(X, y, epochs=50, batch_size=5, verbose=0)
        is_sequence_based = True  # LSTM needs 3D sequences

    elif best_model == 'GRU':
        model = build_gru(look_back)
        model.fit(X, y, epochs=50, batch_size=5, verbose=0)
        is_sequence_based = True

    elif best_model == 'CNN-LSTM':
        model = build_cnn_lstm(look_back)
        model.fit(X, y, epochs=50, batch_size=5, verbose=0)
        is_sequence_based = True

    elif best_model == 'RandomForest' or best_model == 'XGBoost':
        X_train_rf, y_train_rf = create_dataset(store_series.values.reshape(-1, 1), look_back)
        X_train_rf = X_train_rf.reshape(X_train_rf.shape[0], -1)  # Reshape to 2D for tree models
        y_train_rf = y_train_rf
        if best_model == 'RandomForest':
            model = build_random_forest(X_train_rf, y_train_rf)
        else:
            model = build_xgboost(X_train_rf, y_train_rf)
        is_sequence_based = False  # RandomForest/XGBoost needs 2D

    elif best_model == 'Prophet':
        future_store_predictions = build_prophet(store_series)[-future_months:]
        future_predictions[store_name] = future_store_predictions
        continue  # Skip the rest as Prophet is already complete

    else:
        print(f"Unknown model type for store: {store_name}")
        continue

    # Forecast future months for models except Prophet
    last_sequence = scaled_data[-look_back:].reshape(1, look_back, 1) if is_sequence_based else scaled_data[-look_back:].reshape(1, -1)
    future_store_predictions = []
    for _ in range(future_months):
        next_prediction = model.predict(last_sequence)

        if is_sequence_based:
            # For sequence-based models (LSTM, GRU, CNN-LSTM)
            future_store_predictions.append(next_prediction[0, 0])
            # Update the sequence with the new prediction, ensuring dimensions match
            last_sequence = np.append(last_sequence[:, 1:, :], [[next_prediction[0]]], axis=1)
        else:
            # For RandomForest/XGBoost
            future_store_predictions.append(next_prediction[0])
            # Update the last sequence for non-sequence models
            last_sequence = np.append(last_sequence[:, 1:], [next_prediction], axis=1)

    # Inverse transform the future predictions
    future_store_predictions = scaler.inverse_transform(np.array(future_store_predictions).reshape(-1, 1)).flatten()

    # Store the future predictions in a dictionary
    future_predictions[store_name] = future_store_predictions

# Create a new DataFrame to store the predictions in the same format as the profits Excel
future_dates = pd.date_range(start=profits_data.columns[-1], periods=future_months + 1, freq='M')[1:]
future_df = pd.DataFrame(future_predictions, index=future_dates).T

# Add the "Store Name" column back
future_df.insert(0, 'Store Name', future_df.index)

# Save the future predictions DataFrame
future_output_file = 'Future_Store_Predictions.xlsx'
future_df.to_excel(future_output_file, index=False)
print(f"Future predictions saved to {future_output_file}.")
